In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from sklearn.model_selection import train_test_split
from utils.adaptacion import sda_grid_search
from keras.models import load_model

#otros
import os
import time
import pandas as pd
import numpy as np
from sklearn.externals import joblib
import itertools

#variables para guardar los resultados
tipo = pruebas[4]
dataset_name = datasets[1]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [6]:
def split_src_tgt(X_src, X_tgt, test_size=0.2):
    # se divide el dataset para los datos de entrenamiento y validacion del SDA
    X_tr_src, X_val_src, _, _ = train_test_split(X_src, np.zeros(X_src.shape[0]), test_size=0.2, random_state=42)
    
    X_tr_tgt, X_val_tgt, _, _ = train_test_split(X_tgt, np.zeros(X_tgt.shape[0]), test_size=0.2, random_state=42)
    
    X_train = np.concatenate((X_tr_src, X_tr_tgt))
    X_val = np.concatenate((X_val_src, X_val_tgt))
    
    return X_train, X_val

# Adaptación

## Creación de modelos de adaptación.

Por cada par de dominios se entrenan distintos modelos según los parámetros establecidos.
Al entrenar cada modelo, este es inmediatamente probado, siendo almacenado o descartado según su desempeño.

Cada modelo es guardado en la ruta: models/twitter/sda/me2\_[dimensiones]\_[dominio\_fuente]_[dominio\_objetivo].pkl

# Pruebas con el dataset de Twitter (2000 Dimensiones)

In [3]:
dims = 2000

In [4]:
print tipo
print dataset_name
print data_path
print dims

sda
twitter
data
2000


In [5]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [7]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me2_%d_models_paths.pkl' % dims)

parameters = {
    'noises': [0.3 , 0.5, 0.8],
    'layers': [[int(dims/2)], [int(dims/2), int(dims/4)]],
    'epochs': [50, 25],
}

pairs = list(itertools.permutations(domains, 2))
i = 1

#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega
print "Adaptando entre:"
for src, tgt in pairs:
    print "%d) %s - %s" % (i, src, tgt)
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()

    X_train, X_val = split_src_tgt(X_src, X_tgt, test_size=0.2)

    best_autoencoder, best_encoder, score = sda_grid_search(X_train, X_val, X_src, y_src, parameters, n_jobs=4)
    print "\tMejor modelo: %.3f" % score

    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix_ae = "me2_ae_%d_%s_%s.h5" % (dims, src, tgt)
    prefix_e = "me2_e_%d_%s_%s.h5" % (dims, src, tgt)
    
    best_ae_save_path = os.path.join(folder_path, prefix_ae)
    best_e_save_path = os.path.join(folder_path, prefix_e)

    print "\tGuardando autoencoder en %s" % best_ae_save_path
    best_autoencoder.save(best_ae_save_path)

    print "\tGuardando encoder en %s\n" % best_e_save_path
    best_encoder.save(best_e_save_path)

    i = i+1

print "\nAdaptaciones completadas"

Adaptando entre:
1) rio2016 - thevoice
	pr: 0.300 - epochs: 50 - layers: [1000]
	Entrenando autoencoder...
	0.886

	pr: 0.300 - epochs: 25 - layers: [1000]
	Entrenando autoencoder...
	0.889

	pr: 0.300 - epochs: 50 - layers: [1000, 500]
	Entrenando autoencoder...
	0.862

	pr: 0.300 - epochs: 25 - layers: [1000, 500]
	Entrenando autoencoder...
	0.873

	pr: 0.500 - epochs: 50 - layers: [1000]
	Entrenando autoencoder...
	0.880

	pr: 0.500 - epochs: 25 - layers: [1000]
	Entrenando autoencoder...
	0.894

	pr: 0.500 - epochs: 50 - layers: [1000, 500]
	Entrenando autoencoder...
	0.878

	pr: 0.500 - epochs: 25 - layers: [1000, 500]
	Entrenando autoencoder...
	0.834

	pr: 0.800 - epochs: 50 - layers: [1000]
	Entrenando autoencoder...
	0.879

	pr: 0.800 - epochs: 25 - layers: [1000]
	Entrenando autoencoder...
	0.893

	pr: 0.800 - epochs: 50 - layers: [1000, 500]
	Entrenando autoencoder...
	0.870

	pr: 0.800 - epochs: 25 - layers: [1000, 500]
	Entrenando autoencoder...
	0.861

	Mejor modelo: 0.89

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	0.711

	pr: 0.300 - epochs: 25 - layers: [1000]
	Entrenando autoencoder...


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	0.730

	pr: 0.300 - epochs: 50 - layers: [1000, 500]
	Entrenando autoencoder...


...

	0.725

	pr: 0.800 - epochs: 50 - layers: [1000, 500]
	Entrenando autoencoder...


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	0.701

	pr: 0.800 - epochs: 25 - layers: [1000, 500]
	Entrenando autoencoder...


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	0.710

	Mejor modelo: 0.731
	Guardando autoencoder en models/twitter/sda/me2_ae_2000_general_thevoice.h5
	Guardando encoder en models/twitter/sda/me2_e_2000_general_thevoice.h5


Adaptaciones completadas


## Pruebas de clasificación

In [8]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
            
    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### SDA ####
    #############
    # se adaptan los dominios usando SDA
    print "Adaptando dominios..."
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix_e = "me2_e_%d_%s_%s.h5" % (dims, src, tgt)
    best_e_save_path = os.path.join(folder_path, prefix_e)

    best_encoder = load_model(best_e_save_path)


    X_tr = labeled[src]['X_tr'][:, :dims].todense()
    X_tr_a = best_encoder.predict(X_tr)
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    X_ts_a = best_encoder.predict(X_ts)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-clf.score(X_ts_a, y_ts)


    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['SDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i += 1
            
print "\nPruebas completadas."

Tarea 1 de 6
Cargando modelo existente.
Adaptando dominios...


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/keras/models.py:240: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Tarea 2 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 6
Cargando modelo existente.
Adaptando dominios...


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

Tarea 6 de 6
Cargando modelo existente.
Adaptando dominios...


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar


Pruebas completadas.


In [9]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,SDA,r->t,rio2016,thevoice,7.362689,22.206439,14.843750
1,SDA,r->g,rio2016,general,26.998127,43.351740,16.353613
2,SDA,t->r,thevoice,rio2016,10.208711,19.600726,9.392015
3,SDA,t->g,thevoice,general,26.998127,40.611373,13.613246
4,SDA,g->r,general,rio2016,10.208711,23.003630,12.794918
5,SDA,g->t,general,thevoice,7.362689,25.686553,18.323864


In [10]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/twitter/sda/me2_2000.csv
Resultados guardados.


# Pruebas con el dataset de Twitter (1000 Dimensiones)

In [11]:
dims = 1000

In [12]:
print tipo
print dataset_name
print data_path
print dims

sda
twitter
data
1000


In [13]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [14]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me2_%d_models_paths.pkl' % dims)

parameters = {
    'noises': [0.3 , 0.5, 0.8],
    'layers': [[int(dims/2)], [int(dims/2), int(dims/4)]],
    'epochs': [50, 25],
}

pairs = list(itertools.permutations(domains, 2))
i = 1

#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega
print "Adaptando entre:"
for src, tgt in pairs:
    print "%d) %s - %s" % (i, src, tgt)
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()

    X_train, X_val = split_src_tgt(X_src, X_tgt, test_size=0.2)

    best_autoencoder, best_encoder, score = sda_grid_search(X_train, X_val, X_src, y_src, parameters, n_jobs=4)
    print "\tMejor modelo: %.3f" % score

    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix_ae = "me2_ae_%d_%s_%s.h5" % (dims, src, tgt)
    prefix_e = "me2_e_%d_%s_%s.h5" % (dims, src, tgt)
    
    best_ae_save_path = os.path.join(folder_path, prefix_ae)
    best_e_save_path = os.path.join(folder_path, prefix_e)

    print "\tGuardando autoencoder en %s" % best_ae_save_path
    best_autoencoder.save(best_ae_save_path)

    print "\tGuardando encoder en %s\n" % best_e_save_path
    best_encoder.save(best_e_save_path)

    i = i+1

print "\nAdaptaciones completadas"

Adaptando entre:
1) rio2016 - thevoice
	pr: 0.300 - epochs: 50 - layers: [500]
	Entrenando autoencoder...
	0.864

	pr: 0.300 - epochs: 25 - layers: [500]
	Entrenando autoencoder...
	0.871

	pr: 0.300 - epochs: 50 - layers: [500, 250]
	Entrenando autoencoder...
	0.800

	pr: 0.300 - epochs: 25 - layers: [500, 250]
	Entrenando autoencoder...
	0.803

	pr: 0.500 - epochs: 50 - layers: [500]
	Entrenando autoencoder...
	0.877

	pr: 0.500 - epochs: 25 - layers: [500]
	Entrenando autoencoder...
	0.864

	pr: 0.500 - epochs: 50 - layers: [500, 250]
	Entrenando autoencoder...
	0.839

	pr: 0.500 - epochs: 25 - layers: [500, 250]
	Entrenando autoencoder...
	0.840

	pr: 0.800 - epochs: 50 - layers: [500]
	Entrenando autoencoder...
	0.850

	pr: 0.800 - epochs: 25 - layers: [500]
	Entrenando autoencoder...
	0.877

	pr: 0.800 - epochs: 50 - layers: [500, 250]
	Entrenando autoencoder...
	0.821

	pr: 0.800 - epochs: 25 - layers: [500, 250]
	Entrenando autoencoder...
	0.820

	Mejor modelo: 0.877
	Guardando

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	0.641

	pr: 0.300 - epochs: 25 - layers: [500]
	Entrenando autoencoder...


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	0.634

	pr: 0.300 - epochs: 50 - layers: [500, 250]
	Entrenando autoencoder...


...

	0.653

	pr: 0.800 - epochs: 50 - layers: [500, 250]
	Entrenando autoencoder...


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	0.641

	pr: 0.800 - epochs: 25 - layers: [500, 250]
	Entrenando autoencoder...


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

	0.619

	Mejor modelo: 0.657
	Guardando autoencoder en models/twitter/sda/me2_ae_1000_general_thevoice.h5
	Guardando encoder en models/twitter/sda/me2_e_1000_general_thevoice.h5


Adaptaciones completadas


## Pruebas de clasificación

In [15]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
            
    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### SDA ####
    #############
    # se adaptan los dominios usando SDA
    print "Adaptando dominios..."
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix_e = "me2_e_%d_%s_%s.h5" % (dims, src, tgt)
    best_e_save_path = os.path.join(folder_path, prefix_e)

    best_encoder = load_model(best_e_save_path)


    X_tr = labeled[src]['X_tr'][:, :dims].todense()
    X_tr_a = best_encoder.predict(X_tr)
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    X_ts_a = best_encoder.predict(X_ts)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-clf.score(X_ts_a, y_ts)


    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['SDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i += 1
            
print "\nPruebas completadas."

Tarea 1 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 6
Cargando modelo existente.
Adaptando dominios...


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

Tarea 6 de 6
Cargando modelo existente.
Adaptando dominios...


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar


Pruebas completadas.


In [16]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,SDA,r->t,rio2016,thevoice,19.045928,33.333333,14.287405
1,SDA,r->g,rio2016,general,30.407980,42.426350,12.018370
2,SDA,t->r,thevoice,rio2016,7.372958,21.029946,13.656987
3,SDA,t->g,thevoice,general,30.407980,47.830887,17.422907
4,SDA,g->r,general,rio2016,7.372958,23.026316,15.653358
5,SDA,g->t,general,thevoice,19.045928,34.114583,15.068655


In [17]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/twitter/sda/me2_1000.csv
Resultados guardados.
